## 朴素贝叶斯

### 1.基本定义
分类是把一个事物分到某个类别中。一个事物具有很多属性，把它的众多属性看作一个向量，即
$$x=(x1,x2,x3,…,xn)$$
用x这个向量来代表这个事物，x的集合记为X，称为属性集。类别也有很多种，用集合
$$C={c1,c2,…cm}$$
一般X和C的关系是不确定的，可以将X和C看作是随机变量，P(C|X)称为C的后验概率，与之相对的，P(C)称为C的先验概率。
根据贝叶斯公式，后验概率
$$P(C|X)=P(X|C)P(C)/P(X)$$
但在比较不同C值的后验概率时，分母P(X)总是常数，忽略掉，后验概率
$$P(C|X)=P(X|C)P(C)$$
先验概率P(C)可以通过计算训练集中属于每一个类的训练样本所占的比例，对类条件概率P(X|C)的估计，我们只谈论朴素贝叶斯分类器方法，因为朴素贝叶斯假设事物属性之间相互条件独立，
$$P(X|C)=∏P(xi|ci)$$

### 2.模型原理
    朴素贝叶斯分类器是一种有监督学习，常见有两种模型，多项式模型(multinomial model)即为词频型和伯努利模型(Bernoulli model)即文档型，还有一种高斯模型。
    前二者的计算粒度不一样，多项式模型以单词为粒度，伯努利模型以文件为粒度，因此二者的先验概率和类条件概率的计算方法都不同。计算后验概率时，对于一个文档d，多项式模型中，只有在d中出现过的单词，才会参与后验概率计算，伯努利模型中，没有在d中出现，但是在全局单词表中出现的单词，也会参与计算，不过是作为“反方”参与的。

(1) 高斯模型

In [ ]:
# 从sklearn.datasets里导入20类新闻文本数据抓取器。
from sklearn.datasets import fetch_20newsgroups
# 从互联网上即时下载新闻样本,subset='all'参数代表下载全部近2万条文本存储在变量news中。
news = fetch_20newsgroups(subset='all')
 
# 从sklearn.cross_validation导入train_test_split模块用于分割数据集。
from sklearn.model_selection import train_test_split
# 对news中的数据data进行分割，25%的文本用作测试集；75%作为训练集。
X_train, X_test, y_train, y_test = train_test_split(news.data, news.target, test_size=0.25, random_state=33)
 
# 分别使用停用词过滤配置初始化CountVectorizer与TfidfVectorizer。
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
count_filter_vec, tfidf_filter_vec = CountVectorizer(analyzer='word', stop_words='english'), TfidfVectorizer(analyzer='word', stop_words='english')
 
# 使用带有停用词过滤的CountVectorizer对训练和测试文本分别进行量化处理。
X_count_filter_train = count_filter_vec.fit_transform(X_train)
X_count_filter_test = count_filter_vec.transform(X_test)
 
# 使用带有停用词过滤的TfidfVectorizer对训练和测试文本分别进行量化处理。
X_tfidf_filter_train = tfidf_filter_vec.fit_transform(X_train)
X_tfidf_filter_test = tfidf_filter_vec.transform(X_test)

from sklearn.naive_bayes import GaussianNB
mnb_count_filter = GaussianNB()
mnb_count_filter.fit(X_count_filter_train, y_train)
print ('The accuracy of classifying 20newsgroups using Naive Bayes (CountVectorizer by filtering stopwords):', mnb_count_filter.score(X_count_filter_test, y_test))
y_count_filter_predict = mnb_count_filter.predict(X_count_filter_test)
 
# 初始化另一个默认配置的朴素贝叶斯分类器，并对TfidfVectorizer后的数据进行预测与准确性评估。
mnb_tfidf_filter =  GaussianNB()
mnb_tfidf_filter.fit(X_tfidf_filter_train, y_train)
print ('The accuracy of classifying 20newsgroups with Naive Bayes (TfidfVectorizer by filtering stopwords):', mnb_tfidf_filter.score(X_tfidf_filter_test, y_test))
y_tfidf_filter_predict = mnb_tfidf_filter.predict(X_tfidf_filter_test)
 
# 对上述两个模型进行更加详细的性能评估。
from sklearn.metrics import classification_report
print (classification_report(y_test, y_count_filter_predict, target_names = news.target_names))
print (classification_report(y_test, y_tfidf_filter_predict, target_names = news.target_names))

(2)多项式模型 

    在多项式模型中，设某文档d=(t1,t2,…,tk)，tk是该文档中出现过的单词，允许重复，则先验概率P(c)= 类c下单词总数/整个训练样本的单词总数。类条件概率P(tk|c)=(类c下单词tk在各个文档中出现过的次数之和+1)/(类c下单词总数+|V|)。
    其中V是训练样本的单词表（即抽取单词，单词出现多次，只算一个），|V|则表示训练样本包含多少种单词。P(tk|c)可以看作是单词tk在证明d属于类c上提供了多大的证据，而P(c)则可以认为是类别c在整体上占多大比例(有多大可能性)。


In [2]:
from sklearn.naive_bayes import MultinomialNB
mnb_count_filter = MultinomialNB()
mnb_count_filter.fit(X_count_filter_train, y_train)
print ('The accuracy of classifying 20newsgroups using Naive Bayes (CountVectorizer by filtering stopwords):', mnb_count_filter.score(X_count_filter_test, y_test))
y_count_filter_predict = mnb_count_filter.predict(X_count_filter_test)
 
# 初始化另一个默认配置的朴素贝叶斯分类器，并对TfidfVectorizer后的数据进行预测与准确性评估。
mnb_tfidf_filter = MultinomialNB()
mnb_tfidf_filter.fit(X_tfidf_filter_train, y_train)
print ('The accuracy of classifying 20newsgroups with Naive Bayes (TfidfVectorizer by filtering stopwords):', mnb_tfidf_filter.score(X_tfidf_filter_test, y_test))
y_tfidf_filter_predict = mnb_tfidf_filter.predict(X_tfidf_filter_test)
 

from sklearn.metrics import classification_report
print (classification_report(y_test, y_count_filter_predict, target_names = news.target_names))
print (classification_report(y_test, y_tfidf_filter_predict, target_names = news.target_names))

The accuracy of classifying 20newsgroups using Naive Bayes (CountVectorizer by filtering stopwords): 0.8637521222410866
The accuracy of classifying 20newsgroups with Naive Bayes (TfidfVectorizer by filtering stopwords): 0.8826400679117148
                          precision    recall  f1-score   support

             alt.atheism       0.85      0.89      0.87       201
           comp.graphics       0.62      0.88      0.73       250
 comp.os.ms-windows.misc       0.93      0.22      0.36       248
comp.sys.ibm.pc.hardware       0.62      0.88      0.73       240
   comp.sys.mac.hardware       0.93      0.85      0.89       242
          comp.windows.x       0.82      0.85      0.84       263
            misc.forsale       0.90      0.79      0.84       257
               rec.autos       0.91      0.91      0.91       238
         rec.motorcycles       0.98      0.94      0.96       276
      rec.sport.baseball       0.98      0.92      0.95       251
        rec.sport.hockey       0.9

(3)伯努利模型

    P(c)= 类c下文件总数/整个训练样本的文件总数 
    P(tk|c)=(类c下包含单词tk的文件数+1)/(类c下包含的文件+2)


In [3]:
from sklearn.naive_bayes import BernoulliNB
mnb_count_filter = BernoulliNB()
mnb_count_filter.fit(X_count_filter_train, y_train)
print ('The accuracy of classifying 20newsgroups using Naive Bayes (CountVectorizer by filtering stopwords):', mnb_count_filter.score(X_count_filter_test, y_test))
y_count_filter_predict = mnb_count_filter.predict(X_count_filter_test)
 
# 初始化另一个默认配置的朴素贝叶斯分类器，并对TfidfVectorizer后的数据进行预测与准确性评估。
mnb_tfidf_filter = BernoulliNB()
mnb_tfidf_filter.fit(X_tfidf_filter_train, y_train)
print ('The accuracy of classifying 20newsgroups with Naive Bayes (TfidfVectorizer by filtering stopwords):', mnb_tfidf_filter.score(X_tfidf_filter_test, y_test))
y_tfidf_filter_predict = mnb_tfidf_filter.predict(X_tfidf_filter_test)
 

from sklearn.metrics import classification_report
print (classification_report(y_test, y_count_filter_predict, target_names = news.target_names))
print (classification_report(y_test, y_tfidf_filter_predict, target_names = news.target_names))

The accuracy of classifying 20newsgroups using Naive Bayes (CountVectorizer by filtering stopwords): 0.733446519524618
The accuracy of classifying 20newsgroups with Naive Bayes (TfidfVectorizer by filtering stopwords): 0.733446519524618
                          precision    recall  f1-score   support

             alt.atheism       0.89      0.52      0.66       201
           comp.graphics       0.80      0.60      0.69       250
 comp.os.ms-windows.misc       0.80      0.03      0.06       248
comp.sys.ibm.pc.hardware       0.36      0.93      0.52       240
   comp.sys.mac.hardware       0.53      0.92      0.67       242
          comp.windows.x       0.96      0.68      0.79       263
            misc.forsale       0.64      0.90      0.75       257
               rec.autos       0.53      0.92      0.67       238
         rec.motorcycles       0.97      0.88      0.92       276
      rec.sport.baseball       0.90      0.90      0.90       251
        rec.sport.hockey       0.97 

## SVM
支持向量机（Support Vector Machine，SVM）是一个经典两类分类算法，
其找到的分割超平面具有更好的鲁棒性，因此广泛使用在很多任务上，并表现
出了很强优势。
给定一个两类分类器数据集 
$$D = {(x^{(n)}, y^{(n)})}^N_n=1$$
其中 $y_n \in\{+1,-1\}$
如果两类样本是线性可分的，即存在一个超平面
$$w^T +b = 0 $$
将两类样本分开，那么对于每个样本都有
$$y^{(n)}(w^TX^{(n)}+b)>0$$
定义整个数据集D中所有样本到分割超平面的最短距离为间隔（Margin）$\gamma$
$$ \gamma = min_n(\gamma^{(n)})$$
如果间隔γ 越大，其分割超平面对两个数据集的划分越稳定，不容易受噪声等因素影响。

支持向量机的目标是寻找一个超平面$(w^∗, b^∗)$使得$\gamma$ 最大，即
$$max_{w,b} (\gamma)$$


$$ s.t. \quad  \frac { y^{(n)}(w^TX^{(n)}+b)} {||w||} \ge \gamma , \forall n$$
令$||w||· \gamma = 1$，则$max_{w,b} (\gamma)$等价于
$$ max_{w,b}( \frac {1} {||w||^2}) $$ 
$$ s.t. \quad y^{(n)}(w^TX^{(n)}+b) \ge 1,\forall n $$
数据集中所有满足$y^{(n)}(w^TX^{(n)}+b) = 1$的样本点，都称为支持向量（Support
Vector）。
对于一个线性可分的数据集，其分割超平面有很多个，但是间隔最大的超
平面是唯一的。下图给定了支持向量机的最大间隔分割超平面的示例，其红色
样本点为支持向量
![](img/SVM.png)
为了找到最大间隔分割超平面，将目标函数写为凸优化问题
$$ min_{w,b}(\frac {||w||^2}{2})$$
$$ s.t. \quad 1 - y^{(n)}(w^TX^{(n)}+b) \le 0 , \forall n$$
使用拉格朗日乘数法，其拉格朗日函数为 
$$ L(w,b,\lambda) = \frac {||w||^2}{2} + \sum_{n =1}^N \lambda_n ( 1 - y^{(n)}(w^TX^{(n)}+b))$$
其中 $\lambda_1 ≥ 0,..., \lambda_N ≥ 0$ 为拉格朗日乘数。计算 $L(w, b, λ) $关于 w 和 b 的导数，
并令其等于0得到

$$ w = \sum_{n =1 } ^ N \lambda_ny^{(n)}x^{(n)}$$
$$ \sum_{n =1 } ^ N \lambda_ny^{(n)} = 0$$
代入得，拉格朗日对偶函数 
$$ \Gamma(\lambda) = -\frac {1}{2}\sum_{n =1 } ^ N\sum_{m =1 } ^ N \lambda_m \lambda_n y^{(m)} y^{(n)} ( X^{(m)})^T X^{(n)} + \sum_{n =1 } ^ N \lambda_n  $$

支持向量机的主优化问题为凸优化问题，满足强对偶性，即主优化问题可
以通过最大化对偶函数$max\lambda≥0 Γ(\lambda)$来求解。

对偶函数$Γ(\lambda)$是一个凹函数，因
此最大化对偶函数是一个凸优化问题，可以通过多种凸优化方法来进行求解，
得到拉格朗日乘数的最优值$\lambda^*$。但由于其约束条件的数量为训练样本数量，一
般的优化方法代价比较高，因此在实践中通常采用比较高效的优化方法，比如
SMO（Sequential Minimal Optimization）算法[Platt, 1998]等
根据KKT条件中的互补松弛条件，最优解满足 
$$ \lambda_n^*( 1 - y^{(n)}(w^{*T}X^{(n)}+b^*)) = 0 $$
如果样本$ x(n) $不在约束边界上，$\lambda_n^* = 0$，其约束失效；如果样本 x(n) 在约
束边界上， $\lambda_n^* \ge 0$。这些在约束边界上样本点称为支持向量（support vector），
即离决策平面距离最近的点。

在计算出$\lambda^*$ 后，根据公式(3.89)计算出最优权重$w^*$，最优偏置$b^*$ 可以通
过任选一个支持向量$(\overline x,\overline y)$计算得到。
$$ b^* = \overline y - w^{*T}\overline x$$

最优参数的支持向量机的决策函数为
$$f(x) = sgn(w^{*T}X^{(n)}+b^*) \\= sgn(\sum_{n = 1}^ N \lambda_n^* y^{(n)} (X^{(n)})^T X + b^*$$

支持向量机的决策函数只依赖于$\lambda_n^* > 0$的样本点，即支持向量。

支持向量机的目标函数可以通过SMO等优化方法得到全局最优解，因此比
其它分类器的学习效率更高。此外，支持向量机的决策函数只依赖于支持向量，
与训练样本总数无关，分类速度比较快。
### 核函数
支持向量机还有一个重要的优点是可以使用核函数（kernel function）隐式
地将样本从原始特征空间映射到更高维的空间，并解决原始特征空间中的线性
不可分问题。比如在一个变换后的特征空间ϕ中，支持向量机的决策函数为
$$f(x) = sgn(w^{*T} \phi(x)+b^*) \\= sgn(\sum_{n = 1}^ N \lambda_n^* y^{(n)} K(X^{(n)}, X) + b^*$$

其中 $ K(x,z) = \phi(x)^T \phi(z)$为核函数。通常我们不需要显式地给出 $\phi(x)$ 的具体
形式，可以通过核技巧（kernel trick）来构造。比如以$x, z ∈ R^2 $为例，我们可
以构造一个核函数 
$$K(x,z) = (1 + x^Tz)^2 = \phi(x)^T \phi(z)$$
来隐式地计算x, z在特征空间$\phi$中的内积
其中 $ \phi(x) = [ 1, \sqrt 2 x_1,\sqrt 2 x_2,\sqrt 2 x_1x_2,x_1^2,x_2^2]$
### 软间隔 
在支持向量机的优化问题中，约束条件比较严格。如果训练集中的样本在
特征空间中不是线性可分的，就无法找到最优解。为了能够容忍部分不满足约
束的样本，我们可以引入松弛变量$\xi$，将优化问题变为
$$ min_{w,b}(\frac {||w||^2}{2} + C\sum_{n = 1}^N \xi_n)$$
$$ s.t. \quad 1 - y^{(n)}(w^TX^{(n)}+b)-\xi_n \le 0 , \forall n$$
$$ \xi_n \ge 0 ,\forall n$$
其中参数$C > 0$用来控制间隔和松弛变量惩罚的平衡。引入松弛变量的间隔称
为软间隔（soft margin）。公式(3.98)也可以表示为经验风险+正则化项的形式。
$$ min_{w,b}(\sum_{n=1}^Nmax(0,1 - y^{(n)}(w^TX^{(n)}+b))+ \frac {1}{C}\frac {||w||^2}{2}) $$

其中$ max(0,1 - y^{(n)}(w^TX^{(n)}+b))$称为hinge损失函数 ,
$\frac {1}{C}$ 可以看作是正则化系数。
软间隔支持向量机的参数学习和原始支持向量机类似，其最终决策函数也
只和支持向量有关，即满足$\quad 1 - y^{(n)}(w^TX^{(n)}+b)-\xi_n = 0$的样本。 
### 不同线性模型对比 
![](img/model.png)
![](img/loss.png)


In [14]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn import datasets

sess = tf.Session()

# 加载数据
# iris.data = [(Sepal Length, Sepal Width, Petal Length, Petal Width)]
iris = datasets.load_iris()
x_vals = np.array([[x[0], x[3]] for x in iris.data])
y_vals = np.array([1 if y == 0 else -1 for y in iris.target])
# 分离训练和测试集
train_indices = np.random.choice(len(x_vals),
                                 round(len(x_vals)*0.8),
                                 replace=False)
test_indices = np.array(list(set(range(len(x_vals))) - set(train_indices)))
x_vals_train = x_vals[train_indices]
x_vals_test = x_vals[test_indices]
y_vals_train = y_vals[train_indices]
y_vals_test = y_vals[test_indices]
batch_size = 100

# 初始化feedin
x_data = tf.placeholder(shape=[None, 2], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)

# 创建变量
A = tf.Variable(tf.random_normal(shape=[2, 1]))
b = tf.Variable(tf.random_normal(shape=[1, 1]))

# 定义线性模型
model_output = tf.subtract(tf.matmul(x_data, A), b)

# Declare vector L2 'norm' function squared
l2_norm = tf.reduce_sum(tf.square(A))

# Loss = max(0, 1-pred*actual) + alpha * L2_norm(A)^2
alpha = tf.constant([0.01])
classification_term = tf.reduce_mean(tf.maximum(0., tf.subtract(1., tf.multiply(model_output, y_target))))
loss = tf.add(classification_term, tf.multiply(alpha, l2_norm))
# 训练
my_opt = tf.train.GradientDescentOptimizer(0.01)
train_step = my_opt.minimize(loss)

init = tf.global_variables_initializer()
sess.run(init)

# Training loop
loss_vec = []
train_accuracy = []
test_accuracy = []
for i in range(20000):
    rand_index = np.random.choice(len(x_vals_train), size=batch_size)
    rand_x = x_vals_train[rand_index]
    rand_y = np.transpose([y_vals_train[rand_index]])
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
[[a1], [a2]] = sess.run(A)
[[b]] = sess.run(b)
slope = -a2/a1
y_intercept = b/a1
best_fit = []

x1_vals = [d[1] for d in x_vals]

for i in x1_vals:
    best_fit.append(slope*i+y_intercept)


# Separate I. setosa
setosa_x = [d[1] for i, d in enumerate(x_vals) if y_vals[i] == 1]
setosa_y = [d[0] for i, d in enumerate(x_vals) if y_vals[i] == 1]
not_setosa_x = [d[1] for i, d in enumerate(x_vals) if y_vals[i] == -1]
not_setosa_y = [d[0] for i, d in enumerate(x_vals) if y_vals[i] == -1]

plt.plot(setosa_x, setosa_y, 'o', label='I. setosa')
plt.plot(not_setosa_x, not_setosa_y, 'x', label='Non-setosa')
plt.plot(x1_vals, best_fit, 'r-', label='Linear Separator', linewidth=3)
plt.ylim([0, 10])
plt.legend(loc='lower right')
plt.title('Sepal Length vs Pedal Width')
plt.xlabel('Pedal Width')
plt.ylabel('Sepal Length')
plt.show()

<Figure size 640x480 with 1 Axes>

## LDA

### 介绍
    LDA（Latent Dirichlet Allocation）是一种文档主题生成模型，也称为一个三层贝叶斯概率模型，包含词、主题和文档三层结构。

    LDA是一种非监督机器学习技术，可以用来识别大规模文档集（document collection）或语料库（corpus）中潜藏的主题信息。它采用了词袋（bag of words）的方法，这种方法将每一篇文档视为一个词频向量，从而将文本信息转化为了易于建模的数字信息。

### 算法
    思想：给定训练集样例，设法将样例投影到一条直线上，使得同类样例的投影尽可能接近，异类样例的投影点尽可能原理；在对新的样本进行分类时，将其投影到同样的这条直线上，再根据投影点的位置来确定新样本的类别。
### 假设
    数据呈正态分布
    各类别数据具有相同的协方差矩阵
    样本的特征从统计上来说相互独立
    事实上，即使违背上述假设，LDA仍能正常工作
### 数学原理 
二项分布 与其共轭的 Beta分布 
$$ Binom(k|n,p) = (n \ k)p^k(1-p)^{(n-k)}$$
$$ Beta(p|a,b) = \Gamma (a+b) \Gamma(a) \Gamma(b) P^{(a-1)} (1-P)^{(b-1) }$$
其中 $\Gamma$ 是 Gamma函数 $\Gamma(x) = (x-1)!$
三项的多项分布
$$multi(m1,m2,m3|n,p1,p2,p3)=\frac {n!}{m1!m2!m3!}p_1^{m_1} p_2^{m_2}p_3^{m_3}$$
超过二维的Beta分布我们一般称之为狄利克雷(以下称为Dirichlet )分布。也可以说Beta分布是Dirichlet 分布在二维时的特殊形式。从二维的Beta分布表达式，我们很容易写出三维的Dirichlet分布如下：
$$Dirichlet(p1,p2,p3|a1,a2,a3)=\frac {Γ(a1+a2+a3)}{Γ(a1)Γ(a2)Γ(a3)} p_1^{a_1−1}p_2^{a_1-1}p_3^{a_3 -1}$$
一般意义上的K维Dirichlet 分布表达式为
$$Dirichlet(p⃗ |α⃗ )= \frac {Γ(∑_{k=1}^Kα_k)}{∏^K_{k=1}Γ(α_k)}∏_{k=1}^Kp_k^{a_k−1}k$$
　我们的目标是找到每一篇文档的主题分布和每一个主题中词的分布。在LDA模型中，我们需要先假定一个主题数目K，这样所有的分布就都基于K个主题展开。那么具体LDA模型是怎么样的呢？具体如下图：
![](img/LDA.png)
　LDA假设文档主题的先验分布是Dirichlet分布，即对于任一文档d, 其主题分布θd为：
$$\theta_d=Dirichlet(α⃗ )$$
其中，α为分布的超参数，是一个K维向量。

LDA假设主题中词的先验分布是Dirichlet分布，即对于任一主题k
, 其词分布βk为：
$$β_k=Dirichlet(η⃗ )$$

其中，η
为分布的超参数，是一个V维向量。V
代表词汇表里所有词的个数。

对于数据中任一一篇文档d
中的第n个词，我们可以从主题分布θd中得到它的主题编号zdn的分布为：
$$z_{dn}=multi(θd)$$

而对于该主题编号，得到我们看到的词wdn
的概率分布为： 
$$w_{dn}=multi(βzdn)$$

理解LDA主题模型的主要任务就是理解上面的这个模型。这个模型里，我们有M
个文档主题的Dirichlet分布，而对应的数据有M个主题编号的多项分布，这样
$$(α→θd→z⃗ d)$$
就组成了Dirichlet-multi共轭，可以使用前面提到的贝叶斯推断的方法得到基于Dirichlet分布的文档主题后验分布。

如果在第d个文档中，第k个主题的词的个数为：n(k)d
, 则对应的多项分布的计数可以表示为
$$n⃗ _d=(n(1)d,n(2)d,...n(K)d)$$

利用Dirichlet-multi共轭，得到θd
的后验分布为：
$$Dirichlet(θd|α⃗ +n⃗ d)$$

同样的道理，对于主题与词的分布，我们有K
个主题与词的Dirichlet分布，而对应的数据有K个主题编号的多项分布，这样(η→βk→w⃗ (k)

)就组成了Dirichlet-multi共轭，可以使用前面提到的贝叶斯推断的方法得到基于Dirichlet分布的主题词的后验分布。

如果在第k个主题中，第v个词的个数为：n(v)k
, 则对应的多项分布的计数可以表示为
$$n⃗ _k=(n(1)k,n(2)k,...n(V)k)$$

利用Dirichlet-multi共轭，得到βk
的后验分布为：
$$Dirichlet(βk|η⃗ +n⃗ k)$$